# Library importations

In [1]:
pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=118376 sha256=1f772a9d9303b344b4fda815d34dba9ba553da7a33da8d2da1f1eb67499c61bc
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw
Note: you may need to restart the kernel to use updated packages.


In [52]:
import numpy as np
import pandas as pd
import os
import librosa
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Loading digit pronunciations for references

In [12]:
def load_digits(path):
    zero, sr = librosa.load('{}/0-55-20.wav'.format(path))
    one, sr = librosa.load('{}/1-55-20.wav')
    two, sr = librosa.load('{}/2-55-20.wav')
    three, sr = librosa.load('{}/3-55-20.wav')
    four, sr = librosa.load('{}/4-55-20.wav')
    five, sr = librosa.load('{}/5-55-20.wav')
    six, sr = librosa.load('{}/6-55-20.wav')
    seven, sr = librosa.load('{}/7-55-20.wav')
    eight, sr = librosa.load('{}/8-55-20.wav')
    nine, sr = librosa.load('{}/9-55-20.wav')
    
    return zero, one, two, three, four, five, six, seven, eight, nine

In [13]:
digits = load_digits('/kaggle/input/vocal-digits/digits')

## 2. Dynamic Time Warping (DTW) spoken digit detection

In [53]:
def distance_matrix(signal1, signal2):   
    return cdist(signal1, signal2, "cosine")

In [36]:
def dtw(signal1, signal2):
    distance = distance_matrix(signal1, signal2)
    
    NCOA = 0 #Normalized cost of alignment

    cost = np.zeros((signal1.shape[0]+1, signal2.shape[0]+1))
    
    cost[:,0] = np.inf
    cost[signal1.shape[0],:] = np.inf
    cost[signal1.shape[0],0] = 0
    
    traceback_matrix = np.zeros((signal1.shape[0]+1, signal2.shape[0]+1))
    for i in range(signal1.shape[0], -1, -1):
        for j in range(signal2.shape[0]):
            curr_cost = [cost[i+1][j], cost[i+1][j+1], cost[i][j]]
                        #match,        insertion,      deletion
            action_index = np.argmin(curr_cost)
            traceback_matrix[i][j] = action_index
            cost[i][j+1] = curr_cost[action_index] + distance[i][j]
            
    path = [(i,j)]
    while i < signal1.shape[0]+1 or j < signal2.shape[0]+1:
        if traceback_matrix[i][j] == 0:
            #match
            i -= 1
            j += 1
        elif traceback_matrix[i][j] == 1:
            #insert
            i -= 1
        else:
            #delete
            j += 1
        NCOA += cost[i][j]
        path.append((i,j))
        
    return NCOA, path[1:], cost[:-1, 1:]

In [150]:
def detect_digit(audio_path):
    #Preprocess the input audio
    audio, sr = librosa.load(audio_path)
       
    #Distances to each number pronunciation
    distances_mfcc = []
    mfcc_score = 0
    distances_ms = []
    ms_score = 0
    distances_lpc = []
    lpc_score = 0
    #Threshold for detecting if the audio symbolizes a digit
    digit_threshold = 1
    
    for index, digit in enumerate(digits):
        #distance, path, cost = dtw(audio, digit)
        #MFCC approach
        digit_mfcc = librosa.feature.mfcc(y=digit, n_mfcc=40).T
        audio_mfcc = librosa.feature.mfcc(y=audio, n_mfcc=40).T
        distance, path = fastdtw(audio_mfcc, digit_mfcc, dist=euclidean)
        distances_mfcc.append(distance / len(path))
        if np.argmin(np.asarray(distances_mfcc)) == index:
            mfcc_score += 1
        #MS approach
        digit_ms = np.log(librosa.feature.melspectrogram(y=digit, n_mels=40)).T
        audio_ms = np.log(librosa.feature.melspectrogram(y=audio, n_mels=40)).T
        distance, path = fastdtw(audio_ms, digit_ms, dist=euclidean)
        distances_ms.append(distance / len(path))
        if np.argmin(np.asarray(distances_ms)) == index:
            ms_score += 1 
        #LPC approach
        digit_lpc = librosa.lpc(y=digit, order=40)
        audio_lpc = librosa.lpc(y=audio, order=40)
        distance, path = fastdtw(audio_lpc, digit_lpc)
        distances_lpc.append(distance / len(path))
        if np.argmin(np.asarray(distances_lpc)) == index:
            lpc_score += 1 
        
    #We search for the minimum distance (most likely digit)
    prediction_index = np.argmin(np.asarray(distances_lpc))
    
    #Check if the audio is a digit based on the threshold
    if distances_lpc[prediction_index] > digit_threshold:
        print("The audio doesn't symbolize a digit\n")
    else:
        print("The audio is a digit, most likely the number {}\n".format(prediction_index))

    print("Normalized cost of alignment scores:")
    print("For numbers: {}".format(" ".join(["{:<5}".format(i) for i in range(10)])))
    print("MFCC:        {}".format(" ".join("{:.2f}".format(distances_mfcc[i]) for i in range(10))))
    print("MS:          {}".format(" ".join("{:.2f}".format(distances_ms[i]) for i in range(10))))
    print("LPC:         {}".format(" ".join("{:.2f}".format(distances_lpc[i]) for i in range(10))))
    print("Correct guesses for MFC: {}, MS: {}, LPC: {}".format(mfcc_score, ms_score, lpc_score))

In [149]:
detect_digit('/kaggle/input/test-digits/test_digits/3-51-20.wav')

The audio is a digit, most likely the number 3

Normalized cost of alignment scores:
For numbers: 0     1     2     3     4     5     6     7     8     9    
MFCC:        82.6 77.2 79.4 75.0 81.3 78.0 81.3 78.8 75.8 77.5
MS:          13.8 13.1 13.6 13.0 13.5 13.2 13.5 13.4 13.3 13.3
LPC:         0.6 0.9 0.6 0.7 1.0 0.9 0.6 0.4 0.2 0.8
Correct guesses for MFC: 3, MS: 3, LPC: 4


## 3. Find the vocally closest person

In [159]:
def find_closest_vocal(path):
    person_dict = {}

    distances_lpc = {}
    closest_person = ''
    closest_person_min = np.inf
    
    for filename in os.listdir(path):
        if filename.endswith(".wav"):
            person_identifier = filename.split('-')[1] + "-" + filename.split('-')[2].split('.')[0]

            if person_identifier not in person_dict:
                person_dict[person_identifier] = [librosa.load(path + "/" + filename)[0]]
            else:
                person_dict[person_identifier].append(librosa.load(path + "/" + filename)[0])

    for person, files in person_dict.items():
        for i in range(10):
            #LPC approach
            digit_lpc = librosa.lpc(y=digits[i], order=40)
            audio_lpc = librosa.lpc(y=files[i], order=40)
            distance, path = fastdtw(audio_lpc, digit_lpc)
            if i == 0:
                distances_lpc[person] = distance / len(path)
            else:
                distances_lpc[person] += distance / len(path)
        if distances_lpc[person] < closest_person_min:
            closest_person = person
                
    print("The closest person vocally to you has the index: {}".format(closest_person))

In [160]:
find_closest_vocal('/kaggle/input/test-digits/test_digits')

The closest person vocally to you has the index: 51-20


The strategy we used above to detect the vocally closest person is the following.<br>
We go through all the test data (test_digits) and we compare each of our spoken digits to each of another persons spoken digits<br>
by calculating the costs between them using DTW. For preprocessing we used LPC because we saw the best results with it while<br>
detecting numbers.<br>
We calculate normalized cost of alignment (NCOA) for each digit and add them all up together for each person.<br>
NCOA is calculated as the total_cost_of_path/length_of_path.<br>
At least we just look at which person has the minimum sum because that means that person has the closest pronunciation to us.

## 4. DTW Pruning

As we saw in this project DTW can be very computationally expensive, it's time complaxity is quadratic which can be disastrous for bigger sequences of data. To address this issue a couple of pruning strategies have been advised as solutions such as:<br>

### Sakoe-Chiba Band:

The Sakoe-Chiba band is a restriction on the alignment path, limiting the allowed deviations in the warping path. It introduces a constraint that only allows points within a certain diagonal band to be matched, excluding points outside this band. This significantly reduces the number of possible alignments.

### Itakura Parallelogram:

The Itakura parallelogram is another constraint applied during DTW. It limits the allowed warping path to a parallelogram shape, which is especially useful for speech recognition applications. Similar to the Sakoe-Chiba band, this constraint reduces the search space and computational complexity.

### Early Abandoning:

Early abandoning involves terminating the DTW computation early if it becomes evident that the current path is already worse than the best path found so far. This strategy relies on maintaining a lower bound on the distance during computation and abandoning the alignment if the lower bound exceeds the best distance found.

### Pruning by Threshold:

Pruning by threshold involves setting a distance threshold, below which the algorithm stops the computation of certain alignments. If the accumulated distance exceeds the threshold, the alignment is pruned, reducing the number of comparisons.

### Windowing:

Windowing involves restricting the search space to a specific region around the diagonal. Only points within a certain window are considered for alignment, excluding points outside the window. This is a less strict constraint compared to the Sakoe-Chiba band but still reduces the number of calculations.